## Injection benchmark

### Setup

In [1]:
import sys
import platform
from antidote import __version__, is_compiled
print(f"""
== CPU == 
{platform.processor()} 

== Python ==
{sys.version}

== Antidote ==
{__version__()} {'(cython)' if is_compiled() else ''}
""")


== CPU == 
Intel(R) Core(TM) i7-6600U CPU @ 2.60GHz 

== Python ==
3.9.0 (default, Nov 24 2020, 22:07:31) 
[GCC 9.3.0]

== Antidote ==
0.8.0 (cython)



### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as those functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [2]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

3.15 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
from antidote import world, Service, inject

class Service1(Service):
    pass


class Service2(Service):
    def __init__(self, service1: Service1):
        self.service1 = service1


class Service3(Service):
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2


class Service4(Service):
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

@inject
def f2(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [5]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f(*args)

139 ns ± 2.28 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [6]:
assert f2() == f(*args)
%timeit f2()

539 ns ± 21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [7]:
assert f2(*args) == f(*args)
%timeit f2(*args)

208 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Method call
Overhead when applied on a method

In [8]:
class Dummy:
    def method(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4
    
    @inject
    def method2(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4

dummy = Dummy()

In [9]:
%timeit dummy.method(*args)

212 ns ± 9.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
assert dummy.method(*args) == dummy.method2()
%timeit dummy.method2()

990 ns ± 53 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Non singleton

We compare the overhead of creating the whole service each time.

In [11]:
class ServiceX(Service):
    __antidote__ = Service.Conf(singleton=False)

def g(s: ServiceX):
    return s

@inject
def g2(s: ServiceX):
    return s

In [12]:
%timeit g(ServiceX())

201 ns ± 3.09 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
assert isinstance(g2(), ServiceX)
assert g2() is not g2()
%timeit g2()

680 ns ± 24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation
Cost of creating wired objects

In [14]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
        
class Obj2:
    @inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
    

In [15]:
%timeit Obj(*args)

549 ns ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%timeit Obj2()

1.31 µs ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Factory

In [17]:
from antidote import factory

class ServiceF:
    pass

def service_factory() -> ServiceF:
    return ServiceF()

@factory(singleton=False)
def service_factory2() -> ServiceF:
    return ServiceF()

def h(s):
    return s

@inject(dependencies=(ServiceF @ service_factory2,))
def h2(s):
    return s

In [18]:
%timeit h(service_factory())

285 ns ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
assert isinstance(h2(), ServiceF)
assert h2() is not h2()
%timeit h2()

783 ns ± 118 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Configuration


In [20]:
from antidote import Constants

class Conf(Constants):
    A = 'A'
    B = 'B'

    def get(self, key):
        return key
    
def ff(a, b):
    return a, b

@inject(dependencies=(Conf.A, Conf.B))
def ff2(a, b):
    return a, b

In [21]:
conf = Conf()
%timeit ff(conf.get('A'), conf.get('B'))

394 ns ± 29.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
assert ff(conf.get('A'), conf.get('B')) == ff2()
%timeit ff2()

524 ns ± 17.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Custom Provider

In [23]:
import time
from antidote import world
from antidote.core import Provider, DependencyInstance
dep = object()
dep2 = object()

@world.provider
class SlowProvider(Provider):
    def exists(self, dependency):
        return dependency is dep2
    
    def provide(self, dependency, container):
        time.sleep(.01)
        return DependencyInstance("sleepy")
            

@world.provider
class CustomProvider(Provider):
    def exists(self, dependency):
        return dependency is dep
    
    def provide(self, dependency, container):
        return DependencyInstance("Found it !")


@inject(dependencies=dict(d=dep))
def f_provider(d):
    return d

In [24]:
assert f_provider() == "Found it !"
%timeit f_provider()

1.17 µs ± 65.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
